In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:/Users/HP/OneDrive/Chatbot/project.json').read()
intents = json.loads(data_file)
#C:\Users\HP\OneDrive\Chatbot


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('C:/Users/HP/OneDrive/Chatbot/words.pkl','wb'))
pickle.dump(classes,open('C:/Users/HP/OneDrive/Chatbot/classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Using TensorFlow backend.


77 documents
18 classes ['Ranking', 'about', 'club', 'courses', 'cse', 'exam', 'fee', 'goodbye', 'greeting', 'griet', 'location', 'moodle', 'options', 'phno', 'placements', 'results', 'scholarships', 'thanks']
91 unique lemmatized words ["'s", ',', 'about', 'activity', 'and', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'club', 'college', 'company', 'computer', 'contact', 'could', 'course', 'cse', 'curricular', 'day', 'detail', 'do', 'engineering', 'exam', 'fee', 'for', 'good', 'goodbye', 'griet', 'group', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'is', 'know', 'later', 'located', 'location', 'many', 'me', 'moodle', 'name', 'next', 'nice', 'non', 'notif', 'number', 'of', 'offered', 'phno', 'phone', 'placement', 'provided', 'rank', 'ranking', 'reimbursement', 'result', 'scholarship', 'science', 'see', 'show', 'statistic', 'structure', 'subject', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'up', 'visited', 'w

In [3]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('C:/Users/HP/OneDrive/Chatbot/chatbot_model.h5', hist)

print("model created")

Epoch 1/200
77/77 [==============================] - 2s 28ms/step - loss: 2.8763 - accuracy: 0.0649
Epoch 2/200
77/77 [==============================] - 0s 450us/step - loss: 2.8032 - accuracy: 0.1169
Epoch 3/200
77/77 [==============================] - 0s 508us/step - loss: 2.7137 - accuracy: 0.1818
Epoch 4/200
77/77 [==============================] - 0s 476us/step - loss: 2.6068 - accuracy: 0.2208
Epoch 5/200
77/77 [==============================] - 0s 509us/step - loss: 2.5520 - accuracy: 0.2078
Epoch 6/200
77/77 [==============================] - 0s 560us/step - loss: 2.4406 - accuracy: 0.2468
Epoch 7/200
77/77 [==============================] - 0s 508us/step - loss: 2.2990 - accuracy: 0.3117
Epoch 8/200
77/77 [==============================] - 0s 572us/step - loss: 2.2769 - accuracy: 0.2987
Epoch 9/200
77/77 [==============================] - 0s 515us/step - loss: 2.1386 - accuracy: 0.3636
Epoch 10/200
77/77 [==============================] - 0s 505us/step - loss: 2.0846 - accurac

77/77 [==============================] - 0s 389us/step - loss: 0.1116 - accuracy: 0.9870
Epoch 82/200
77/77 [==============================] - 0s 415us/step - loss: 0.2109 - accuracy: 0.9351
Epoch 83/200
77/77 [==============================] - 0s 402us/step - loss: 0.1234 - accuracy: 0.9740
Epoch 84/200
77/77 [==============================] - 0s 402us/step - loss: 0.1267 - accuracy: 0.9740
Epoch 85/200
77/77 [==============================] - 0s 415us/step - loss: 0.1369 - accuracy: 0.9740
Epoch 86/200
77/77 [==============================] - 0s 415us/step - loss: 0.1653 - accuracy: 0.9481
Epoch 87/200
77/77 [==============================] - 0s 402us/step - loss: 0.1165 - accuracy: 0.9870
Epoch 88/200
77/77 [==============================] - 0s 454us/step - loss: 0.0886 - accuracy: 0.9870
Epoch 89/200
77/77 [==============================] - 0s 415us/step - loss: 0.0760 - accuracy: 0.9870
Epoch 90/200
77/77 [==============================] - 0s 428us/step - loss: 0.1108 - accuracy: 

77/77 [==============================] - 0s 493us/step - loss: 0.0778 - accuracy: 0.9870
Epoch 161/200
77/77 [==============================] - 0s 406us/step - loss: 0.0730 - accuracy: 0.9870
Epoch 162/200
77/77 [==============================] - 0s 398us/step - loss: 0.0224 - accuracy: 1.0000
Epoch 163/200
77/77 [==============================] - 0s 488us/step - loss: 0.0894 - accuracy: 0.9740
Epoch 164/200
77/77 [==============================] - 0s 666us/step - loss: 0.0273 - accuracy: 1.0000
Epoch 165/200
77/77 [==============================] - 0s 427us/step - loss: 0.0781 - accuracy: 0.9610
Epoch 166/200
77/77 [==============================] - 0s 445us/step - loss: 0.0376 - accuracy: 1.0000
Epoch 167/200
77/77 [==============================] - 0s 454us/step - loss: 0.0612 - accuracy: 0.9870
Epoch 168/200
77/77 [==============================] - 0s 431us/step - loss: 0.1253 - accuracy: 0.9481
Epoch 169/200
77/77 [==============================] - 0s 415us/step - loss: 0.0936 - a

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('C:/Users/HP/OneDrive/Chatbot/chatbot_model.h5')
import json
import random
intents = json.loads(open('C:/Users/HP/OneDrive/Chatbot/project.json').read())
words = pickle.load(open('C:/Users/HP/OneDrive/Chatbot/words.pkl','rb'))
classes = pickle.load(open('C:/Users/HP/OneDrive/Chatbot/classes.pkl','rb'))


def callback(event):
    webbrowser.open_new(event.widget.cget("text"))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


